In [21]:

import calendar
months = list(map(lambda x:x.lower(),calendar.month_name))[1:]
import pandas as pd
from pandas import DataFrame

import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import matplotlib.ticker as mtick

import matplotlib.pylab as pylab
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 10),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

import os
project_folder =os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
import warnings
warnings.filterwarnings(action='ignore')
from IPython.display import display
from IPython.display import Markdown
from IPython.display import display_html




In [2]:
# That's not part of the procedure
pd.read_csv(f'{project_folder}/data/process/new_data.csv').reset_index().iloc[0].to_csv('output_2/fileds_to_work_on.csv')

In [9]:
# show the questions
data_questions = pd.read_csv(f'{project_folder}/data/process/new_data.csv', skiprows=[2]).reset_index().iloc[0]
data_questions

index                                                          0
StartDate                                             Start Date
EndDate                                                 End Date
Status                                             Response Type
IPAddress                                             IP Address
                                     ...                        
Q38            What is the highest level of education you hav...
Q39            What do you consider your main ethnic origin o...
Q39_10_TEXT    What do you consider your main ethnic origin o...
Q40            Where did you hear about this survey? - Select...
Q40_3_TEXT     Where did you hear about this survey? - Other ...
Name: 0, Length: 75, dtype: object

#### Data Preparation

In [3]:
data_general = pd.read_csv(f'{project_folder}/data/process/final_geo_coding.csv', skiprows=[1, 2]).reset_index()
# Read dictionary of relevant questions
questions_to_examine=pd.read_csv(f'output_2/fileds_to_work_on_updated.csv')
rel_questions_dic = dict(zip(questions_to_examine['Number'], questions_to_examine['Abbreviation ']))
field_to_work_on = 'index'

In [4]:
special_questions = ['Q26', 'Q27', 'Q32', 'Q33']


In [108]:
questions_to_examine_1  = questions_to_examine.set_index('Number')

In [135]:
html_table = questions_to_examine_1.to_html()
text ="<span style='color:blue; font-size:20px'>The following questions will be the main focus of our analysis. We will begin by looking at each question individually.</span>"
display(Markdown(text))
# display_html(html_table, raw=True)
questions_to_examine.set_index('Number')

<span style='color:blue; font-size:20px'>The following questions will be the main focus of our analysis. We will begin by looking at each question individually.</span>

,Question,Abbreviation
Number,,
Q25,Did you replace the stolen bicycle or bicycle ...,Is replaced
Q26,How many days did it take to recover or replac...,Time replaced
Q27,"After recovering your bike, how many additiona...",Time rideable
Q28_1,"Over the past year, how often did you cycle (w...",Fall habit
Q28_2,"Over the past year, how often did you cycle (w...",Winter habit
Q28_3,"Over the past year, how often did you cycle (w...",Spring habit
Q28_4,"Over the past year, how often did you cycle (w...",Summer habit
Q29,What was the main purpose of the trips you mad...,Purpose trip
Q30,Thinking about the trips you made with the sto...,Mode switch


#### Q26 - How many days did it take to recover or replace your stolen bicycle or part?

In [5]:
data_general['Q26'] = data_general['Q26'].apply(lambda x: str(x).rstrip().lstrip())
gb_data_0= data_general.groupby('Q26').count()[field_to_work_on].sort_values(ascending=False)
# Values to delete
null_values =['nan','N/a','Na','na','None','-','Months']
gb_data = gb_data_0[~gb_data_0.index.isin(null_values)]

In [6]:
from qu_26_27 import InterpretTime
time_q26 = InterpretTime(gb_data)
for res in time_q26.new_gb_data.iterrows():
    time_q26.main_part(res)

In [55]:
def calc_statistics(class_data:DataFrame):
    r"""
    the class count the number of respondents for the number of days
    :param class_data:
    :return: list of date groupby the number of days, statistics and count and percentage of those with doesn't recover/replace/fix atc. their bicycles
    """
    # remove records without number of days
    num_of_days = class_data['num_of_days']
    final_data = class_data[(num_of_days!='') & (num_of_days !=-1)]
    # group by number of days
    mygroupby = final_data.groupby('num_of_days').sum().sort_values(by = 'count',ascending=False)

     # flat @mygroupby to calculate statistic
    l= [[item[0]]*item[1]['count'] for item in mygroupby.iterrows()]
    flat_list = [item for sublist in l for item in sublist]
    df = DataFrame(flat_list)


    # the number of -1
    no_data = class_data[num_of_days ==-1].sum()['count']
    no_dat_dic = (no_data ,no_data/ (len(flat_list)+no_data)*100)
    return mygroupby, df.describe(),{'no_action_yet':no_dat_dic},df


In [120]:
q= questions_to_examine_1.loc['Q26'][0]
res = calc_statistics(time_q26.new_gb_data)
flat_data = res[-1]
day_25 = int(res[1].loc['25%'][0])
day_50 = int(res[1].loc['50%'][0])
more_365  = len(flat_data[flat_data[0]>365])
more_385_percentage = int(round(more_365/len(flat_data)*100,0))
max= int(res[1].loc['max'][0])
no_recoverd_replace =int(round(list(res[2].values())[0][1]))
text =f"<span style='color:blue; font-size:20px'>{q}</span>"
display(Markdown(text))
text = f"<span style='color:blue; font-size:20px'>25% recovered or replaced their bicycles within {day_25} days and 50% within {day_50} days.  {more_385_percentage}% are recovered or replaced after more then 1 year. The longest time  reported to recovered/replaced was {max} days. Among all the participants that answers to this question {no_recoverd_replace}% have not recovered/replaces their bicycles.</span>"
display(Markdown(text))
text = "<span style='color:red; font-size:20px'>An optional analysis would be to compare the results between group members who were replaced and those who recovered. Are there any statistical differences between the two groups?"
display(Markdown(text))

<span style='color:blue; font-size:20px'>How many days did it take to recover or replace your stolen bicycle or part?</span>

<span style='color:blue; font-size:20px'>25% recovered or replaced their bicycles within 20 days and 50% within 49 days.  4% are recovered or replaced after more then 1 year. The longest time  reported to recovered/replaced was 1460 days. Among all the participants that answers to this question 5% have not recovered/replaces their bicycles.</span>

<span style='color:red; font-size:20px'>An optional analysis would be to compare the results between group members who were replaced and those who recovered. Are there any statistical differences between the two groups?

#### Q27 - After recovering your bike, how many additional days did it take before your bicycle was repaired and rideable?

In [89]:
gb_data_first = data_general.groupby('Q27').count()[field_to_work_on].sort_values(ascending=False)
gb_data_27 = gb_data_first.dropna()
time_q27 = InterpretTime(gb_data_27)
for res in time_q27.new_gb_data.iterrows():
    time_q27.main_part(res)
res = calc_statistics(time_q27.new_gb_data)

In [121]:
q= questions_to_examine_1.loc['Q27'][0]
flat_data = res[-1]
day_25 = int(res[1].loc['25%'][0])
day_50 = int(res[1].loc['50%'][0])
more_30  = len(flat_data[flat_data[0]>30])
more_30_percentage = int(round(more_30/len(flat_data)*100,0))
max= int(res[1].loc['max'][0])
no_recoverd_replace =int(round(list(res[2].values())[0][1]))
text =f"<span style='color:blue; font-size:20px'>{q}</span>"
display(Markdown(text))
text = f"<span style='color:blue; font-size:20px'>25% repaired their bicycles within {day_25} days and 50% within {day_50} days. {more_30_percentage}% are repaired after more then 1 month. The longest time  reported to repair was {max} days. Among all the participants that answers to this question {no_recoverd_replace}% have not recovered/replaces their bicycles.</span>"
display(Markdown(text))


<span style='color:blue; font-size:20px'>After recovering your bike, how many additional days did it take before your bicycle was repaired and rideable?</span>

<span style='color:blue; font-size:20px'>25% repaired their bicycles within 20 days and 50% within 49 days. 54% are repaired after more then 1 month. The longest time  reported to repair was 1460 days. Among all the participants that answers to this question 5% have not recovered/replaces their bicycles.</span>

In [172]:
# code for Q32 and Q33 which is multiple choices questions
res=[]
for question_num in special_questions[-2:]:
    print(question_num)
    # Read choice options
    choice_df = pd.read_csv(f'output_2/{question_num}.csv').set_index('question')
    # grpoupby data by the responses to the question_num
    gb_data= data_general.groupby(question_num).count()[field_to_work_on].sort_values(ascending=False).reset_index()
    # Go over each separate response
    for  item in choice_df.iterrows():
        question = item[0]
        # search the current response in the multiple choices responses
        for row_0 in gb_data.iterrows():
            row = row_0[1]
            if question in row[question_num]:
                # if the response was found add the number of respondants that use that respomse as part on 1 or combination of choices
                choice_df.loc[question]+=row[field_to_work_on]
    # calculate percentage
    sumy = choice_df.sum()
    choice_df['per'] = round(choice_df/sumy *100,0)
    choice_df.sort_values(by='per',ascending=False,inplace=True)
    res.append(choice_df)
    # Plot results
    # choice_df['per'].plot(kind='bar',title=rel_questions_dic[question_num])

    # plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0,decimals=0))
    # plt.show()

Q32


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 18: invalid start byte

In [126]:
res

[                                                    count   per
 question                                                       
 I couldn’t recover my bicycle and didn’t replac...    307  25.0
 There might not be a safe place to park my bicy...    216  18.0
 Became accustomed to travelling with a differen...    118  10.0
 Other (please specify):                               125  10.0
 My new bicycle isn’t as capable or comfortable ...    113   9.0
 I don’t have a safe place to store my bicycle a...     94   8.0
 High-security locks or secure parking locations...     89   7.0
 My cycling equipment/accessories/gear (e.g., pa...     75   6.0
 Not riding has made me too out of shape to make...     47   4.0
 My bicycle was damaged and couldn’t be repaired...     11   1.0
 Don’t know/not sure                                    17   1.0,
                                                     count   per
 question                                                       
 My new bicycle was more

In [137]:
q= questions_to_examine_1.loc['Q32'][0]
most_pop = res[0].iloc[0]
text =f"<span style='color:blue; font-size:20px'>{q}</span>"
display(Markdown(text))
text = f"<span style='color:blue; font-size:20px'>the answer with the highest  frequency is:</span>"
display(Markdown(text))
most_pop
q= questions_to_examine_1.loc['Q33'][0]
most_pop = res[1].iloc[0]
text =f"<span style='color:blue; font-size:20px'>{q}</span>"
display(Markdown(text))
text = f"<span style='color:blue; font-size:20px'>the answer with the highest  frequency is:</span>"
display(Markdown(text))
most_pop
text = "<span style='color:red; font-size:20px'>An optional analysis would be to group choices and see the frequency and cross tabulation with demographic and bike type"
display(Markdown(text))

<span style='color:blue; font-size:20px'>What were the main reasons you cycled less often or not at all? (Select up to 3) - Selected Choice</span>

<span style='color:blue; font-size:20px'>the answer with the highest  frequency is:</span>

count    307.0
per       25.0
Name: I couldn’t recover my bicycle and didn’t replace it due to cost or other reason, dtype: float64

<span style='color:blue; font-size:20px'>What were the main reasons you cycled more often? (Select up to 3) - Selected Choice</span>

<span style='color:blue; font-size:20px'>the answer with the highest  frequency is:</span>

count    49.0
per      26.0
Name: My new bicycle was more capable, comfortable to ride, or better suited to my travel needs, dtype: float64

<span style='color:red; font-size:20px'>An optional analysis would be to group choices and see the frequency and cross tabulation with demographic and bike type

In [187]:
res = []
for question in rel_questions_dic.items():
    question_num = question[0]
    if question_num in special_questions:
        continue
    gb_data= data_general.groupby(question_num).count()[field_to_work_on].rename('count').reset_index()
    sumy = gb_data['count'].sum()
    gb_data['per'] = round(gb_data['count']/sumy *100,0)
    gb_data.sort_values(by='per',ascending=False,inplace=True)



    res.append(gb_data)
    # Draw the resuils:
    # gb_data['per'].plot(kind='bar',title=question[1])
    # plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0,decimals=0))
    # plt.show()
res

[                                                 Q25  count   per
 2  ​Yes, and I replaced it with exactly what was ...    828  46.0
 0                                                 No    558  31.0
 1     Yes, but the replacement was something cheaper    426  24.0,
                     Q28_1  count   per
 1         1-3 days a week    593  34.0
 2   4 or more days a week    518  30.0
 0        1-3 days a month    307  18.0
 5                   Never    154   9.0
 4  Less than once a month    143   8.0
 3     Don't know/not sure     16   1.0,
                     Q28_2  count   per
 5                   Never    433  25.0
 1         1-3 days a week    377  22.0
 2   4 or more days a week    358  21.0
 0        1-3 days a month    277  16.0
 4  Less than once a month    239  14.0
 3     Don't know/not sure     16   1.0,
                     Q28_3  count   per
 1         1-3 days a week    587  34.0
 2   4 or more days a week    548  32.0
 0        1-3 days a month    316  18.0
 5       

In [192]:
q = questions_to_examine_1.loc['Q25'][0]
res_25 = res[0]
more_ex = int(res_25[res_25['Q25'] =='\u200bYes, and I replaced it with exactly what was stolen or something more expensive ']['per'].values[0])
cheaper = int(res_25[res_25['Q25'] =='Yes, but the replacement was something cheaper']['per'].values[0])
no= int(res_25[res_25['Q25'] =='No']['per'].values[0])
text = f"<span style='color:blue; font-size:20px'>{q}</span>"
display(Markdown(text))
text = f"<span style='color:blue; font-size:20px'>{no}% did not replace their stolen bicycle, {cheaper}% did replace with cheaper bicycle and {more_ex}%  with more expensive bicycle.</span>"
display(Markdown(text))


# most_pop
# q = questions_to_examine_1.loc['Q33'][0]
# most_pop = res[1].iloc[0]
# text = f"<span style='color:blue; font-size:20px'>{q}</span>"
# display(Markdown(text))
# text = f"<span style='color:blue; font-size:20px'>the answer with the highest  frequency is:</span>"
# display(Markdown(text))
# most_pop
# text = "<span style='color:red; font-size:20px'>An optional analysis would be to group choices and see the frequency and cross tabulation with demographic and bike type"
# display(Markdown(text))

<span style='color:blue; font-size:20px'>Did you replace the stolen bicycle or bicycle part?</span>

<span style='color:blue; font-size:20px'>31% did not replace their stolen bicycle, 24% did replace with cheaper bicycle and 46%  with more expensive bicycle.</span>